# TD2 : Denormalization

In [30]:
import pandas as pd
import json

# Load data from CSV files into DataFrames
employees_df = pd.read_csv('Data\\employees.csv', header=None)
departments_df = pd.read_csv('Data\\departments.csv', header=None)
dept_emp_df = pd.read_csv('Data\\dept_emp.csv', header=None)
dept_manager_df = pd.read_csv('Data\\dept_manager.csv', header=None)
titles_df = pd.read_csv('Data\\titles.csv', header=None)
salaries_df = pd.read_csv('Data\\salaries.csv', header=None)



In [31]:
employees_df

,0,1,2,3,4,5
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


In [33]:
employees_df.columns=['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date']
departments_df.columns=['dept_no','dept_name']
dept_emp_df.columns=['emp_no', 'dept_no', 'from_date', 'to_date']
dept_manager_df.columns=['dept_no', 'emp_no', 'from_date', 'to_date']
titles_df.columns=['emp_no', 'title', 'from_date', 'to_date']
salaries_df.columns=['emp_no','salary', 'from_date', 'to_date']

## Employee-Centric Denormalization

In [34]:
dept_emp_new=pd.merge(dept_emp_df, departments_df, on='dept_no', how='left')
dept_emp_new.head()

,emp_no,dept_no,from_date,to_date,dept_name
0,10001,d005,1986-06-26,9999-01-01,Development
1,10002,d007,1996-08-03,9999-01-01,Sales
2,10003,d004,1995-12-03,9999-01-01,Production
3,10004,d004,1986-12-01,9999-01-01,Production
4,10005,d003,1989-09-12,9999-01-01,Human Resources


In [35]:
dept_manager_new=pd.merge(dept_manager_df, departments_df, on='dept_no', how='left')
dept_manager_new.head()

,dept_no,emp_no,from_date,to_date,dept_name
0,d001,110022,1985-01-01,1991-10-01,Marketing
1,d001,110039,1991-10-01,9999-01-01,Marketing
2,d002,110085,1985-01-01,1989-12-17,Finance
3,d002,110114,1989-12-17,9999-01-01,Finance
4,d003,110183,1985-01-01,1992-03-21,Human Resources


In [36]:
# Create a dictionary to store employee-centric data
employee_data = {}

In [37]:
# Iterate through employees
for _, employee in employees_df.iterrows():
    emp_no = employee['emp_no']
    employee_data[emp_no] = {
        'emp_no': emp_no,
        'first_name': employee['first_name'],
        'last_name': employee['last_name'],
        'gender': employee['gender'],
        'birth_date': employee['birth_date'],
        'hire_date': employee['hire_date'],
        'departments': [],
        'titles': [],
        'salaries': []
    }

In [38]:
# Add department information
for _, dept_emp in dept_emp_new.iterrows():
    emp_no = dept_emp['emp_no']
    dept_no = dept_emp['dept_no']
    dept_name=dept_emp['dept_name']
    from_date = dept_emp['from_date']
    to_date = dept_emp['to_date']
    
    if emp_no in employee_data:
        employee_data[emp_no]['departments'].append({
            'dept_no': dept_no,
            'dept_name': dept_name,
            'from_date': from_date,
            'to_date': to_date
        })

In [39]:
# Add manager information
for _, dept_manager in dept_manager_new.iterrows():
    emp_no = dept_manager['emp_no']
    dept_no = dept_manager['dept_no']
    dept_name=dept_manager['dept_name']
    from_date = dept_manager['from_date']
    to_date = dept_manager['to_date']
    
    if emp_no in employee_data:
        employee_data[emp_no]['departments'].append({
            'dept_no': dept_no,
            'dept_name':dept_name,
            'from_date': from_date,
            'to_date': to_date,
            'is_manager': True
        })


In [40]:
# Add title information
for _, title in titles_df.iterrows():
    emp_no = title['emp_no']
    title_name = title['title']
    from_date = title['from_date']
    to_date = title['to_date']
    
    if emp_no in employee_data:
        employee_data[emp_no]['titles'].append({
            'title': title_name,
            'from_date': from_date,
            'to_date': to_date
        })

In [41]:
# Add salary information
for _, salary in salaries_df.iterrows():
    emp_no = salary['emp_no']
    salary_amount = salary['salary']
    from_date = salary['from_date']
    to_date = salary['to_date']
    
    if emp_no in employee_data:
        employee_data[emp_no]['salaries'].append({
            'salary': salary_amount,
            'from_date': from_date,
            'to_date': to_date
        })

In [42]:
# Convert the employee data dictionary to a list of employee records
employee_records = list(employee_data.values())

# Export the data to a JSON file
with open("employee_data4.json", 'w') as json_file:
    json.dump(employee_records, json_file)

print("Data denormalized and exported to employee_data.json.")



Data denormalized and exported to employee_data.json.


In [46]:
# Read the file content
with open('employee_data4.json', 'r') as file:
    content = file.read()

# Replace the target string
content_modified = content.replace('"dept_name": NaN', '"dept_name": "Customer Service"')

# Write the modified content to a new file
with open('employee_data5_modified.json', 'w') as file:
    file.write(content_modified)


In [44]:
len(data)

NameError: name 'data' is not defined

## Simple Department Centric Denormalization

In [ ]:
dept_emp_new2=pd.merge(dept_emp_df, employees_df, on='emp_no', how='left')
dept_emp_new2.head()

,emp_no,dept_no,from_date,to_date,birth_date,first_name,last_name,gender,hire_date
0,10002,d007,1996-08-03,9999-01-01,1964-06-02,Bezalel,Simmel,F,1985-11-21
1,10003,d004,1995-12-03,9999-01-01,1959-12-03,Parto,Bamford,M,1986-08-28
2,10004,d004,1986-12-01,9999-01-01,1954-05-01,Chirstian,Koblick,M,1986-12-01
3,10005,d003,1989-09-12,9999-01-01,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
4,10006,d005,1990-08-05,9999-01-01,1953-04-20,Anneke,Preusig,F,1989-06-02


In [12]:
dept_manager_new2=pd.merge(dept_manager_df, employees_df, on='emp_no', how='left')
dept_manager_new2.head()

,dept_no,emp_no,from_date,to_date,birth_date,first_name,last_name,gender,hire_date
0,d001,110039,1991-10-01,9999-01-01,1963-06-21,Vishwani,Minakawa,M,1986-04-12
1,d002,110085,1985-01-01,1989-12-17,1959-10-28,Ebru,Alpin,M,1985-01-01
2,d002,110114,1989-12-17,9999-01-01,1957-03-28,Isamu,Legleitner,F,1985-01-14
3,d003,110183,1985-01-01,1992-03-21,1953-06-24,Shirish,Ossenbruggen,F,1985-01-01
4,d003,110228,1992-03-21,9999-01-01,1958-12-02,Karsten,Sigstam,F,1985-08-04


In [13]:
department_data={}

In [14]:
for _, department in departments_df.iterrows():
    dept_no = department['dept_no']
    department_data[dept_no] = {
        'dept_no': dept_no,
        'dept_name': department['dept_name'],
        'employees': [],
        'managers': []
    }

In [15]:
for _, dept_emp in dept_emp_df.iterrows():
    emp_no = dept_emp['emp_no']
    dept_no = dept_emp['dept_no']
    from_date = dept_emp['from_date']
    to_date = dept_emp['to_date']
    
    if dept_no in department_data:
        department_data[dept_no]['employees'].append({
            'emp_no': emp_no,
            'from_date': from_date,
            'to_date': to_date
        })

In [16]:
for _, dept_manager in dept_manager_df.iterrows():
    emp_no = dept_manager['emp_no']
    dept_no = dept_manager['dept_no']
    from_date = dept_manager['from_date']
    to_date = dept_manager['to_date']
    
    if dept_no in department_data:
        department_data[dept_no]['managers'].append({
            'emp_no': emp_no,
            'from_date': from_date,
            'to_date': to_date
        })

In [17]:
department_records = list(department_data.values())

with open('department_data.json', 'w') as json_file:
    json.dump(department_records, json_file, indent=2)

print("Data denormalized and exported to department_data.json.")

Data denormalized and exported to department_data.json.


## Department-centric Denormalization

In [18]:
department_data={}
for _, department in departments_df.iterrows():
    dept_no = department['dept_no']
    department_data[dept_no] = {
        'dept_no': dept_no,
        'dept_name': department['dept_name'],
        'employees': [],
        'managers': []  # Initialize the 'managers' list here
    }

In [19]:
#for _, department in departments_df.iterrows():
    #dept_no = department['dept_no']
    #department_data[dept_no] = {
     #   'dept_no': dept_no,
      #  'dept_name': department['dept_name'],
       # 'employees': []
    #}

In [20]:
for _, employee in employees_df.iterrows():
    emp_no = employee['emp_no']
    emp_data = {
        'emp_no': emp_no,
        'first_name': employee['first_name'],
        'last_name': employee['last_name'],
        'gender': employee['gender'],
        'birth_date': employee['birth_date'],
        'hire_date': employee['hire_date'],
        'titles': [],
        'salaries': []
    }
    
    titles_for_employee = titles_df[titles_df['emp_no'] == emp_no]
    for _, title_record in titles_for_employee.iterrows():
        emp_data['titles'].append({
            'title': title_record['title'],
            'from_date': title_record['from_date'],
            'to_date': title_record['to_date']
        })
    
    salaries_for_employee = salaries_df[salaries_df['emp_no'] == emp_no]
    for _, salary_record in salaries_for_employee.iterrows():
        emp_data['salaries'].append({
            'salary': salary_record['salary'],
            'from_date': salary_record['from_date'],
            'to_date': salary_record['to_date']
        })
        
    # Check if the employee is also a manager
    is_manager = len(dept_manager_df[(dept_manager_df['emp_no'] == emp_no)]) > 0
    
    # Add the employee data to the appropriate department
    emp_departments = dept_emp_df[dept_emp_df['emp_no'] == emp_no]
    for _, emp_department in emp_departments.iterrows():
        dept_no = emp_department['dept_no']
        if dept_no in department_data:
            if is_manager:
                department_data[dept_no]['managers'].append(emp_data)
            else:
                department_data[dept_no]['employees'].append(emp_data)

KeyboardInterrupt: 

In [ ]:
department_data

{'d005': {'dept_no': 'd005',
  'dept_name': 'Development',
  'employees': [],
  'managers': [{'emp_no': 110511,
    'first_name': 'DeForest',
    'last_name': 'Hagimont',
    'gender': 'M',
    'birth_date': '1957-07-08',
    'hire_date': '1985-01-01',
    'titles': [{'title': 'Manager',
      'from_date': '1985-01-01',
      'to_date': '1992-04-25'},
     {'title': 'Technique Leader',
      'from_date': '1992-04-25',
      'to_date': '9999-01-01'}],
    'salaries': [{'salary': 48626,
      'from_date': '1985-01-01',
      'to_date': '1986-01-01'},
     {'salary': 49148, 'from_date': '1986-01-01', 'to_date': '1987-01-01'},
     {'salary': 49877, 'from_date': '1987-01-01', 'to_date': '1988-01-01'},
     {'salary': 51690, 'from_date': '1988-01-01', 'to_date': '1988-12-31'},
     {'salary': 52342, 'from_date': '1988-12-31', 'to_date': '1989-12-31'},
     {'salary': 54300, 'from_date': '1989-12-31', 'to_date': '1990-12-31'},
     {'salary': 56962, 'from_date': '1990-12-31', 'to_date': '199

In [ ]:
department_records = list(department_data.values())

with open('department_data3.json', 'w') as json_file:
    json.dump(department_data, json_file)

print("Data denormalized and exported to department_data.json.")

Data denormalized and exported to department_data.json.


In [ ]:
department_data.values()

## Department-centric denormalization based on current job

In [26]:
department_data={}

In [27]:
current_date = '9999-01-01'

In [28]:
for _, department in departments_df.iterrows():
    dept_no = department['dept_no']
    department_data[dept_no] = {
        'dept_no': dept_no,
        'dept_name': department['dept_name'],
        'employees': []
    }

In [ ]:
for _, employee in employees_df.iterrows():
    emp_no = employee['emp_no']
    emp_data = {
        'emp_no': emp_no,
        'first_name': employee['first_name'],
        'last_name': employee['last_name'],
        'titles': [],
        'salaries': []
    }
    
    current_dept = dept_emp_df[(dept_emp_df['emp_no'] == emp_no) & 
                               (dept_emp_df['to_date'] >= current_date)]
    
    if not current_dept.empty:
        dept_no = current_dept.iloc[0]['dept_no']
        if dept_no in department_data:
            # Add titles for the employee
            titles_for_employee = titles_df[(titles_df['emp_no'] == emp_no) & 
                                            (titles_df['to_date'] >= current_date)]
            for _, title_record in titles_for_employee.iterrows():
                emp_data['titles'].append({
                    'title': title_record['title'],
                    'from_date': title_record['from_date'],
                    'to_date': title_record['to_date']
                })

            # Add salaries for the employee
            salaries_for_employee = salaries_df[(salaries_df['emp_no'] == emp_no) & 
                                                (salaries_df['to_date'] >= current_date)]
            for _, salary_record in salaries_for_employee.iterrows():
                emp_data['salaries'].append({
                    'salary': salary_record['salary'],
                    'from_date': salary_record['from_date'],
                    'to_date': salary_record['to_date']
                })

            department_data[dept_no]['employees'].append(emp_data)

In [ ]:
department_records = list(department_data.values())

# Export the data to a JSON file (or your preferred storage)
with open('C:/Users/rosel/Desktop/Structures_de_données_cloud/department_data_current.json', 'w') as json_file:
    json.dump(department_records, json_file, indent=2)

print("Department-centric denormalization created and exported.")